In [1]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import pi,h
from qutip import *
import functools
import operator

Using matplotlib backend: TkAgg


# Two-level qubit with SFQ pulse train

## Using a SFQ pulse train in H1(t)

In [2]:
# check a Gaussian pulse train

# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

t1 = np.linspace(0, n * 2*pi/w_01, 100001)
y1 = functools.reduce(operator.add, [ np.exp(-(t1-tc -i*2*pi/w_01)**2/2/tau**2) * (np.heaviside(t1-i*2*pi/w_01,0)- np.heaviside(t1-2*tc-i*2*pi/w_01,0)) for i in range(n)])

plt.plot(t1, y1)

In [3]:
def H1_coeff(t, args):
    out = functools.reduce(operator.add, 
        [np.exp(-(t-args['tc']-i*2*pi/args['w_01'])**2/2/args['tau']**2) 
         * (np.heaviside(t-i*2*pi/args['w_01'],0) + np.heaviside(t+2*args['tc']-i*2*pi/args['w_01'],0)) 
         for i in range(args['n'])])
    
    return out

In [4]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

# solve Lindblad master equation
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay()
H = [H0, [H1, H1_coeff]]

psi0 = basis(2,0)
tlist_pulseTrain = np.linspace(0, n * 2*pi/w_01, 501)
args = {'tau':tau, 'tc': tc, 'w_01': w_01,'n':n}

output = mesolve(H, psi0, tlist_pulseTrain, [], [sigmaz()], args, progress_bar=True)  

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist_pulseTrain, output.expect[0])
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

10.2%. Run time:   1.77s. Est. time left: 00:00:00:15
20.2%. Run time:   3.65s. Est. time left: 00:00:00:14
30.1%. Run time:   5.20s. Est. time left: 00:00:00:12
40.1%. Run time:   6.64s. Est. time left: 00:00:00:09
50.1%. Run time:   8.21s. Est. time left: 00:00:00:08
60.1%. Run time:   9.74s. Est. time left: 00:00:00:06
70.1%. Run time:  11.21s. Est. time left: 00:00:00:04
80.0%. Run time:  12.68s. Est. time left: 00:00:00:03
90.0%. Run time:  14.38s. Est. time left: 00:00:00:01
Total run time:  15.88s


## Using evolution operator : faster

In [5]:
def H1_coeff_1(t, args):
    """ Time-dependant coefficient of H1 Hamiltonian """
    return np.exp(-t**2/2/args['tau']**2)

def U_f(t):
    """ free time evolution operator """
    return (1j*w_01*t/2*sigmaz()).expm() 

In [7]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 180

# Compute evolution operator over one unit of SFQ pulse train,i.e., one SFQ + free evolotuion
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay() 
H = [H0, [H1, H1_coeff_1]]

tlist = np.linspace(-tc, tc, 101)
args = {'tau':tau, 'tc':tc, 'w_01':w_01}

U_single_Gauss = propagator(H, tlist, [], args=args) # time evolution during a Gaussian pulse

U_single = U_f(2*pi/w_01-tc) * U_single_Gauss[-1] * U_f(-tc) # Eq.26 in PRApplied paper.

U_G = [U_single]
for i in range(n-1):
    U_G.append(U_single*U_G[-1])
    
sigmaz_expect = expect(sigmaz(), U_G*basis(2,0)) # expectation value of sigma_z

# plot
tlist = [2*pi/w_01*i for i in range(n)]
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, sigmaz_expect)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

## Fidelity vs n
As in Fig.4